# imports from Python 3

In [278]:
from __future__ import print_function

# CONSTANTS

In [279]:
# Database Access
DB_CONNECT =\
    'jdbc:postgresql://chicagoboothanalytics.clloyfrn47sj.us-west-1.rds.amazonaws.com:5432/ChicagoBoothAnalytics?user=BoothAnalytics&password=123BACmoderator'

# Database Table Names
APP_NAME =\
    'ChicagoBoothAnalytics_app'
DB_TABLE_NAME___USER =\
    'auth_user'
DB_TABLE_NAME___BUSINESSSECTOR =\
    '"%s_businesssector"' % APP_NAME
DB_TABLE_NAME___GEOGREGION =\
    '"%s_geogregion"' % APP_NAME
DB_TABLE_NAME___ORG =\
    '"%s_org"' % APP_NAME
DB_TABLE_NAME___ORG_BUSINESSSECTOR_M2M =\
    '"%s_org_business_sectors"' % APP_NAME
DB_TABLE_NAME___ORG_GEOGREGION_M2M =\
    '"%s_org_geog_regions"' % APP_NAME
DB_TABLE_NAME___USERINTERSTEDINORGS_M2M =\
    '"%s_userinterestedinorgs_orgs"' % APP_NAME
DB_TABLE_NAME___USERINTERSTEDINORGS =\
    '"%s_userinterestedinorgs"' % APP_NAME
DB_TABLE_NAME___PERSON =\
    '"%s_person"' % APP_NAME
DB_TABLE_NAME___ROLELVEL =\
    '"%s_rolelevel"' % APP_NAME
DB_TABLE_NAME___ROLE =\
    '"%s_role"' % APP_NAME
DB_TABLE_NAME___CAREEROPPORTUNITY_GEOGREGION_M2M =\
    '"%s_careeropportunity_geog_regions"' % APP_NAME
DB_TABLE_NAME___CAREEROPPORTUNITY =\
    '"%s_careeropportunity"' % APP_NAME
DB_TABLE_NAME___USERCAREEROPPORTUNITYSTATUS =\
    '"%s_usercareeropportunitystatus"' % APP_NAME
DB_TABLE_NAME___FACTTYPE =\
    '"%s_facttype"' % APP_NAME
DB_TABLE_NAME___ORGFACT =\
    '"%s_orgfact"' % APP_NAME

NB_ROWS_TO_SHOW = 9

# import Spark; initiate SparkContext & HiveContext

In [280]:
if 'pyspark' not in vars():
    from findspark import init
    init()
    import pyspark
    sc = pyspark.SparkContext()
    hc = pyspark.sql.HiveContext(sc)
    
print('SparkContext: %s' % str(sc))
print('HiveContext: %s' % str(hc))

SparkContext: <pyspark.context.SparkContext object at 0x106eb8ed0>
HiveContext: <pyspark.sql.context.HiveContext object at 0x10848f1d0>


# ORIGINAL DATABASE TABLES

## "User" table

In [281]:
User_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___USER)\
    .select(
        'id',
        'username',
        'first_name',
        'last_name',
        'email')\
    .sort('username')

User_DDF.registerTempTable('User')

User_DDF.cache()

User_DDF.toPandas()

,id,username,first_name,last_name,email
0,2,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu
1,1,admin,,,BoothAnalyticsClub@gmail.com


## "BusinessSector" table

In [282]:
BusinessSector_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___BUSINESSSECTOR)\
    .sort('name')

BusinessSector_DDF.registerTempTable('BusinessSector')

BusinessSector_DDF.cache()

BusinessSector_DDF.toPandas()

,id,name
0,28,Advertising
1,21,Automotive
2,20,Beverages
3,1,Consulting
4,16,Consumer Goods
5,25,Creative
6,7,E-Commerce
7,29,Education
8,31,Entertainment
9,13,Financial Services


## "GeogRegion" table

In [283]:
GeogRegion_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___GEOGREGION)\
    .sort('name')

GeogRegion_DDF.registerTempTable('GeogRegion')

GeogRegion_DDF.cache()

GeogRegion_DDF.toPandas()

,id,name
0,8,"Atlanta, GA"
1,13,"Austin, TX"
2,14,"Boston (Greater), MA"
3,6,"Boston, MA"
4,9,"Chicago (Greater), IL"
5,2,"Chicago, IL"
6,11,"Cincinnati, OH"
7,12,"Los Angeles (Greater), LA"
8,10,"Los Angeles, CA"
9,5,"New York City, NY"


## "Org" table

In [284]:
Org_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___ORG)\
    .sort('name')

Org_DDF.registerTempTable('Org')

Org_DDF.cache()

Org_DDF.show(NB_ROWS_TO_SHOW)

+---+---------+
| id|     name|
+---+---------+
| 83|       3M|
| 84|   Abbott|
| 46|Accenture|
| 44|   Acxiom|
|  8| AdYapper|
|  4|   Adatao|
|  7|    Adobe|
|  9|   Affirm|
| 53|   AirBnB|
+---+---------+
only showing top 9 rows



## "Org_BusinessSector" table

In [285]:
Org_BusinessSector_m2m_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___ORG_BUSINESSSECTOR_M2M)

Org_BusinessSector_m2m_DDF.registerTempTable('Org_BusinessSector_m2m')

Org_BusinessSector_m2m_DDF.show(NB_ROWS_TO_SHOW)

+---+------+-----------------+
| id|org_id|businesssector_id|
+---+------+-----------------+
|  1|     3|                4|
|  2|     3|                7|
|  4|     4|                4|
|  6|     5|                4|
|  8|     6|                4|
|  9|     7|                4|
| 10|     8|                4|
| 11|     8|                5|
| 12|     9|                9|
+---+------+-----------------+
only showing top 9 rows



In [286]:
Org_BusinessSector_DDF = hc.sql(
    'SELECT \
        Org_BusinessSector_m2m.id as id, \
        Org_BusinessSector_m2m.org_id as org_id, \
        Org_BusinessSector_m2m.businesssector_id as businesssector_id, \
        Org.name as org, \
        BusinessSector.name as business_sector \
    FROM \
        Org_BusinessSector_m2m \
        LEFT JOIN Org ON Org_BusinessSector_m2m.org_id = Org.id \
        LEFT JOIN BusinessSector on Org_BusinessSector_m2m.businesssector_id = BusinessSector.id')\
    .sort('org', 'business_sector')

Org_BusinessSector_DDF.registerTempTable('Org_BusinessSector')

Org_BusinessSector_DDF.cache()

Org_BusinessSector_DDF.show(NB_ROWS_TO_SHOW)

+---+------+-----------------+---------+---------------+
| id|org_id|businesssector_id|      org|business_sector|
+---+------+-----------------+---------+---------------+
|110|    83|               25|       3M|       Creative|
|111|    84|               26|   Abbott|     Healthcare|
|112|    84|               27|   Abbott| Pharmaceutical|
| 68|    46|                1|Accenture|     Consulting|
|113|    44|               28|   Acxiom|    Advertising|
|114|    44|                5|   Acxiom|      Marketing|
| 11|     8|                5| AdYapper|      Marketing|
| 10|     8|                4| AdYapper|     Technology|
|  4|     4|                4|   Adatao|     Technology|
+---+------+-----------------+---------+---------------+
only showing top 9 rows



## "Org_GeogRegion" table

In [287]:
Org_GeogRegion_m2m_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___ORG_GEOGREGION_M2M)

Org_GeogRegion_m2m_DDF.registerTempTable('Org_GeogRegion_m2m')

Org_GeogRegion_m2m_DDF.show(NB_ROWS_TO_SHOW)

+---+------+-------------+
| id|org_id|geogregion_id|
+---+------+-------------+
|  1|     3|            1|
|  2|     3|            4|
|  3|     4|            4|
|  4|     5|            4|
|  5|     6|            3|
|  6|     7|            4|
|  7|     7|            7|
|  8|     9|            3|
|  9|    10|            2|
+---+------+-------------+
only showing top 9 rows



In [288]:
Org_GeogRegion_DDF = hc.sql(
    'SELECT \
        Org_GeogRegion_m2m.id as id, \
        Org_GeogRegion_m2m.org_id as org_id, \
        Org_GeogRegion_m2m.geogregion_id as geogregion_id, \
        Org.name as org, \
        GeogRegion.name as geog_region \
    FROM \
        Org_GeogRegion_m2m \
        LEFT JOIN Org ON Org_GeogRegion_m2m.org_id = Org.id \
        LEFT JOIN GeogRegion ON Org_GeogRegion_m2m.geogregion_id = GeogRegion.id')\
    .sort('org', 'geog_region')

Org_GeogRegion_DDF.registerTempTable('Org_GeogRegion')

Org_GeogRegion_DDF.cache()

Org_GeogRegion_DDF.show(NB_ROWS_TO_SHOW)

+---+------+-------------+--------+--------------------+
| id|org_id|geogregion_id|     org|         geog_region|
+---+------+-------------+--------+--------------------+
| 70|    44|            9|  Acxiom|Chicago (Greater)...|
|  3|     4|            4|  Adatao|San Francisco Bay...|
|  7|     7|            7|   Adobe|  Salt Lake City, UT|
|  6|     7|            4|   Adobe|San Francisco Bay...|
|  8|     9|            3|  Affirm|   San Francisco, CA|
| 53|    53|            3|  AirBnB|   San Francisco, CA|
|  9|    10|            2|Allstate|         Chicago, IL|
| 38|    38|            4|Alphabet|San Francisco Bay...|
|  2|     3|            4|  Amazon|San Francisco Bay...|
+---+------+-------------+--------+--------------------+
only showing top 9 rows



## "UserInterestedInOrgs" table

In [289]:
UserInterestedInOrgs_m2m_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___USERINTERSTEDINORGS_M2M)

UserInterestedInOrgs_m2m_DDF.registerTempTable('UserInterestedInOrgs_m2m')

UserInterestedInOrgs_m2m_DDF.show(NB_ROWS_TO_SHOW)

+---+-----------------------+------+
| id|userinterestedinorgs_id|org_id|
+---+-----------------------+------+
|128|                      1|     4|
|129|                      1|     5|
|130|                      1|     7|
|131|                      1|    20|
|132|                      1|    22|
|133|                      1|    26|
|134|                      1|    32|
|135|                      1|    34|
|136|                      1|    39|
+---+-----------------------+------+
only showing top 9 rows



In [290]:
UserInterestedInOrgs_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___USERINTERSTEDINORGS)

UserInterestedInOrgs_DDF.registerTempTable('UserInterestedInOrgs')

UserInterestedInOrgs_DDF = hc.sql(
    'SELECT \
        UserInterestedInOrgs.id as id, \
        UserInterestedInOrgs.user_id as user_id, \
        UserInterestedInOrgs_m2m.org_id as org_id, \
        User.username as username, \
        User.first_name as user_first_name, \
        User.last_name as user_last_name, \
        User.email as user_email_address, \
        Org.name as org_name \
    FROM \
        UserInterestedInOrgs \
        JOIN UserInterestedInOrgs_m2m \
            ON UserInterestedInOrgs.id = UserInterestedInOrgs_m2m.userinterestedinorgs_id \
        JOIN User ON UserInterestedInOrgs.user_id = User.id \
        JOIN Org ON UserInterestedInOrgs_m2m.org_id = Org.id')\
    .sort(
        'username',
        'org_name')

UserInterestedInOrgs_DDF.registerTempTable('UserInterestedInOrgs')

UserInterestedInOrgs_DDF.cache()

UserInterestedInOrgs_DDF.toPandas()

,id,user_id,org_id,username,user_first_name,user_last_name,user_email_address,org_name
0,1,2,4,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,Adatao
1,1,2,7,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,Adobe
2,1,2,53,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,AirBnB
3,1,2,95,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,Box
4,1,2,108,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,Cloudera
5,1,2,20,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,Facebook
6,1,2,103,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,GitHub
7,1,2,39,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,Google
8,1,2,5,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,H2O
9,1,2,22,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,IBM


## "Person" table

In [291]:
Person_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___PERSON)\
    .selectExpr(
        'id',
        'CONCAT( \
            COALESCE( \
                IF(first_name_alias = "", NULL, first_name_alias), \
                first_name), \
            " ", \
            UPPER(last_name)) \
            as name',
        'last_name',
        'first_name',
        'first_name_alias',
        'IF(gender, "male", "female") as gender')\
    .sort('last_name')

Person_DDF.registerTempTable('Person')

Person_DDF.cache()

Person_DDF.show(NB_ROWS_TO_SHOW)

+---+--------------+---------+--------------+----------------+------+
| id|          name|last_name|    first_name|first_name_alias|gender|
+---+--------------+---------+--------------+----------------+------+
| 21|  Paul ARONSON|  Aronson|          Paul|                |  male|
|  2|   Sahil BAHRI|    Bahri|         Sahil|                |  male|
| 18|  Anju BHALDAR|  Bhaldar|          Anju|                |female|
| 16|Rudi BONAPARTE|Bonaparte|          Rudi|                |  male|
| 25|     Greg BUSH|     Bush|       Gregory|            Greg|  male|
|  8|    Cheng CHEN|     Chen|         Cheng|                |  male|
| 11| John CIASULLI| Ciasulli|    John David|            John|  male|
|  7|Amanda COLLINS|  Collins|Amanda Marlene|          Amanda|female|
| 12|       Tony DO|       Do|         Thong|            Tony|  male|
+---+--------------+---------+--------------+----------------+------+
only showing top 9 rows



# "RoleLevel" table

In [292]:
RoleLevel_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___ROLELVEL)\
    .sort('level_number_from_high_to_low')

RoleLevel_DDF.registerTempTable('RoleLevel')
    
RoleLevel_DDF.show(NB_ROWS_TO_SHOW)

+---+------+-----------------------------+
| id| level|level_number_from_high_to_low|
+---+------+-----------------------------+
|  2| Board|                            1|
|  1|     C|                            2|
|  3|senior|                            3|
|  4|   mid|                            4|
|  5|junior|                            5|
+---+------+-----------------------------+



## "Role" table

In [293]:
Role_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___ROLE)

Role_DDF.registerTempTable('Role')

Role_DDF = hc.sql(
    'SELECT \
        Role.id as id, \
        CONCAT(Role.title, " [", RoleLevel.level, "-level]") as title, \
        RoleLevel.level_number_from_high_to_low as level \
    FROM \
        Role \
        JOIN RoleLevel ON Role.level_id = RoleLevel.id')\
    .sort('level_number_from_high_to_low', 'title')

Role_DDF.registerTempTable('Role')

Role_DDF.cache()
    
Role_DDF.show(NB_ROWS_TO_SHOW)

+---+--------------------+-----+
| id|               title|level|
+---+--------------------+-----+
|  2|Chair of the Boar...|    1|
|  5|Director of the B...|    1|
|  8|(Co-)Founder [C-l...|    2|
| 19|Chief Analytics O...|    2|
|  1|Chief Executive O...|    2|
| 20|Chief Financial O...|    2|
| 21|Chief Operating O...|    2|
| 18|Chief Technology ...|    2|
| 27|Director of Analy...|    3|
+---+--------------------+-----+
only showing top 9 rows



## "CareerOpportunity_GeogRegion" table

In [294]:
CareerOpportunity_GeogRegion_m2m_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___CAREEROPPORTUNITY_GEOGREGION_M2M)

CareerOpportunity_GeogRegion_m2m_DDF.registerTempTable('CareerOpportunity_GeogRegion_m2m')

CareerOpportunity_GeogRegion_m2m_DDF.show(NB_ROWS_TO_SHOW)

+---+--------------------+-------------+
| id|careeropportunity_id|geogregion_id|
+---+--------------------+-------------+
|  1|                   5|            2|
|  2|                   3|            3|
|  3|                   1|            3|
|  4|                   2|            4|
|  5|                   4|            3|
|  6|                   6|            1|
|  7|                   6|            4|
|  9|                   7|            4|
| 12|                   9|            4|
+---+--------------------+-------------+
only showing top 9 rows



In [295]:
CareerOpportunity_GeogRegion_DDF = hc.sql(
    'SELECT \
        CareerOpportunity_GeogRegion_m2m.careeropportunity_id as careeropportunity_id, \
        GeogRegion.name as geog_region \
    FROM \
        CareerOpportunity_GeogRegion_m2m \
        LEFT JOIN GeogRegion ON CareerOpportunity_GeogRegion_m2m.geogregion_id = GeogRegion.id')

CareerOpportunity_GeogRegion_DDF.registerTempTable('CareerOpportunity_GeogRegion')

CareerOpportunity_GeogRegion_DDF.cache()

CareerOpportunity_GeogRegion_DDF.show(NB_ROWS_TO_SHOW)

+--------------------+--------------------+
|careeropportunity_id|         geog_region|
+--------------------+--------------------+
|                   5|         Chicago, IL|
|                   3|   San Francisco, CA|
|                   1|   San Francisco, CA|
|                   2|San Francisco Bay...|
|                   4|   San Francisco, CA|
|                   6|         Seattle, WA|
|                   6|San Francisco Bay...|
|                   7|San Francisco Bay...|
|                   9|San Francisco Bay...|
+--------------------+--------------------+
only showing top 9 rows



In [296]:
CareerOpportunity_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___CAREEROPPORTUNITY)

CareerOpportunity_DDF.registerTempTable('CareerOpportunity')

CareerOpportunity_DDF = hc.sql(
    'SELECT \
        CareerOpportunity.id as id, \
        Org.id as org_id, \
        Org.name as org, \
        Role.title as role, \
        Role.level as role_level, \
        CareerOpportunity_GeogRegion.geog_region as geog_region, \
        CareerOpportunity.open as open, \
        CareerOpportunity.posting_date as posting_date, \
        CareerOpportunity.url as url \
    FROM \
        CareerOpportunity \
        LEFT JOIN Org ON CareerOpportunity.org_id = Org.id \
        LEFT JOIN Role ON CareerOpportunity.role_id = Role.id \
        LEFT JOIN CareerOpportunity_GeogRegion ON \
            CareerOpportunity.id = CareerOpportunity_GeogRegion.careeropportunity_id')\
    .sort(
        'org',
        'role_level',
        'role',
        'geog_region')\
    .sort('open', ascending=False)

CareerOpportunity_DDF.registerTempTable('CareerOpportunity')

CareerOpportunity_DDF.cache()

CareerOpportunity_DDF.show(NB_ROWS_TO_SHOW)

+---+------+--------------------+--------------------+----------+--------------------+----+------------+--------------------+
| id|org_id|                 org|                role|role_level|         geog_region|open|posting_date|                 url|
+---+------+--------------------+--------------------+----------+--------------------+----+------------+--------------------+
|  8|     4|              Adatao|Data Scientist [m...|         4|San Francisco Bay...|true|  2015-10-12|http://www.linked...|
| 21|     7|               Adobe|Data Scientist [m...|         4|San Francisco Bay...|true|  2015-10-06|                    |
|  6|     1|Amazon Web Servic...|Machine Learning ...|         4|San Francisco Bay...|true|        null|                    |
|  6|     1|Amazon Web Servic...|Machine Learning ...|         4|         Seattle, WA|true|        null|                    |
|  2|    41|               Banjo|Senior Data Scien...|         4|San Francisco Bay...|true|  2015-09-19|https://www.ka

## "UserCareerOpportunityStatus" table

In [297]:
UserCareerOpportunityStatus_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___USERCAREEROPPORTUNITYSTATUS)
    
UserCareerOpportunityStatus_DDF.registerTempTable('UserCareerOpportunityStatus')

UserCareerOpportunityStatus_DDF = hc.sql(
    'SELECT \
        UserCareerOpportunityStatus.id as id, \
        UserCareerOpportunityStatus.user_id as user_id, \
        UserCareerOpportunityStatus.career_opportunity_id as careeropportunity_id, \
        User.username as username, \
        User.first_name as user_first_name, \
        User.last_name as user_last_name, \
        User.email as user_email, \
        UserCareerOpportunityStatus.status as status \
    FROM \
        UserCareerOpportunityStatus \
        JOIN User ON UserCareerOpportunityStatus.user_id = User.id')

UserCareerOpportunityStatus_DDF.registerTempTable('UserCareerOpportunityStatus')

UserCareerOpportunityStatus_DDF.cache()

UserCareerOpportunityStatus_DDF.toPandas()

,id,user_id,careeropportunity_id,username,user_first_name,user_last_name,user_email,status
0,1,2,21,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,applied; LinkedIn messaged to recruiter; TODO:...
1,2,2,20,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,applied
2,3,2,26,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,applied
3,4,2,29,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,applied; to talk to Luan's friend
4,5,2,31,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,applied; to find recruiter
5,6,2,32,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,Applied
6,7,2,33,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,Initiated convers; no visa sponsorship
7,8,2,17,MBALearnsToCode,Vinh,Luong,MBALearnsToCode@UChicago.edu,Applied; heard nothing


## "FactType" table

In [298]:
FactType_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___FACTTYPE)\
    .sort('label')
    
FactType_DDF.registerTempTable('FactType')

FactType_DDF.toPandas()

,id,label
0,3,LinkedIn profile URL
1,6,business school
2,5,business/work phone number
3,8,careers website
4,1,email address
5,2,hobby
6,4,website
7,7,year founded


## "OrgFact" table

In [299]:
OrgFact_DDF = hc.read.jdbc(
    url=DB_CONNECT,
    table=DB_TABLE_NAME___ORGFACT)

OrgFact_DDF.registerTempTable('OrgFact')

OrgFact_DDF = hc.sql(
    'SELECT \
        OrgFact.id as id, \
        OrgFact.org_id as org_id, \
        OrgFact.fact_type_id as facttype_id, \
        Org.name as org, \
        FactType.label as fact_type, \
        OrgFact.fact as fact \
    FROM \
        OrgFact \
        LEFT JOIN Org ON OrgFact.org_id = Org.id \
        LEFT JOIN FactType ON OrgFact.fact_type_id = FactType.id')\
    .sort(
        'org',
        'fact_type',
        'fact')

OrgFact_DDF.registerTempTable('OrgFact')

OrgFact_DDF.cache()

OrgFact_DDF.show(NB_ROWS_TO_SHOW)

+---+------+-----------+---------+------------+-------------+
| id|org_id|facttype_id|      org|   fact_type|         fact|
+---+------+-----------+---------+------------+-------------+
| 85|    83|          4|       3M|     website|       3M.com|
| 86|    84|          4|   Abbott|     website|   Abbott.com|
| 47|    46|          4|Accenture|     website|Accenture.com|
| 45|    44|          4|   Acxiom|     website|   Acxiom.com|
| 14|     8|          4| AdYapper|     website| AdYapper.com|
|  1|     4|          4|   Adatao|     website|   Adatao.com|
|  4|     4|          7|   Adatao|year founded|         2012|
| 13|     7|          4|    Adobe|     website|    Adobe.com|
| 39|     9|          4|   Affirm|     website|   Affirm.com|
+---+------+-----------+---------+------------+-------------+
only showing top 9 rows



# DERIVED TABLES

## "OrgOverview" table

In [300]:
# OrgFact_website table
FactType_Website_DDF = hc.sql(
    'SELECT \
        id, \
        label \
    FROM \
        FactType \
    WHERE \
        label = "website"')

OrgFact_Website_DDF =\
    OrgFact_DDF.join(
        FactType_Website_DDF,
        on=OrgFact_DDF.facttype_id == FactType_Website_DDF.id)\
    .select(
        'org_id',
        'fact')
    
OrgFact_website_DDF.registerTempTable('OrgFact_Website')


# OrgFact_CareersWebsite table

FactType_CareersWebsite_DDF = hc.sql(
    'SELECT \
        id, \
        label \
    FROM \
        FactType \
    WHERE \
        label = "careers website"')

OrgFact_CareersWebsite_DDF =\
    OrgFact_DDF.join(
        FactType_CareersWebsite_DDF,
        on=OrgFact_DDF.facttype_id == FactType_CareersWebsite_DDF.id)\
    .select(
        'org_id',
        'fact')
    
OrgFact_CareersWebsite_DDF.registerTempTable('OrgFact_CareersWebsite')


# OrgNbOpenCareerOpportunities
OrgNbOpenCareerOpportunities_DDF = hc.sql(
    'SELECT \
        org_id, \
        COUNT(*) as nb_open_career_opportunities \
    FROM \
        CareerOpportunity \
    WHERE \
        open = true \
    GROUP BY \
       org_id')

OrgNbOpenCareerOpportunities_DDF.registerTempTable('OrgNbOpenCareerOpportunities')


# OrgOverview table
OrgOverview_DDF = hc.sql(
    'SELECT \
        Org.id as id, \
        Org.name as name, \
        COLLECT_SET(OrgFact_Website.fact) as websites, \
        COLLECT_SET(OrgFact_CareersWebsite.fact) as careers_websites, \
        COLLECT_SET(Org_BusinessSector.business_sector) as business_sectors, \
        COLLECT_SET(Org_GeogRegion.geog_region) as geog_regions, \
        OrgNbOpenCareerOpportunities.nb_open_career_opportunities as nb_open_career_opportunities \
    FROM \
        Org \
        LEFT JOIN OrgFact_Website ON Org.id = OrgFact_Website.org_id \
        LEFT JOIN OrgFact_CareersWebsite ON Org.id = OrgFact_CareersWebsite.org_id \
        LEFT JOIN Org_BusinessSector ON Org.id = Org_BusinessSector.org_id \
        LEFT JOIN Org_GeogRegion ON Org.id = Org_GeogRegion.org_id \
        LEFT JOIN OrgNbOpenCareerOpportunities ON Org.id = OrgNbOpenCareerOpportunities.org_id \
    GROUP BY \
        Org.id, \
        Org.name, \
        OrgNbOpenCareerOpportunities.nb_open_career_opportunities')\
    .sort('name')

OrgOverview_DDF.toPandas()

,id,name,websites,careers_websites,business_sectors,geog_regions,nb_open_career_opportunities
0,83,3M,[3M.com],[],[Creative],[],NaN
1,84,Abbott,[Abbott.com],[],"[Healthcare, Pharmaceutical]",[],NaN
2,46,Accenture,[Accenture.com],[],[Consulting],[],NaN
3,44,Acxiom,[Acxiom.com],[],"[Advertising, Marketing]","[Chicago (Greater), IL]",NaN
4,8,AdYapper,[AdYapper.com],[],"[Marketing, Technology]",[],NaN
5,4,Adatao,[Adatao.com],[],[Technology],"[San Francisco Bay Area, CA]",1
6,7,Adobe,[Adobe.com],[],[Technology],"[Salt Lake City, UT, San Francisco Bay Area, CA]",1
7,9,Affirm,[Affirm.com],[],"[Financial Services: Consumer Finance, Technol...","[San Francisco, CA]",NaN
8,53,AirBnB,[AirBnB.com],[],"[Hospitality, Technology]","[San Francisco, CA]",NaN
9,85,Alcatel-Lucent,[Alcatel-Lucent.com],[],[Technology],[],NaN


## "MyCareerOpportunity" tables

In [301]:
InterestingOrg_CareersWebsites_DDF = hc.sql(
    'SELECT \
        UserInterestedInOrgs.org_name as name, \
        OrgFact_CareersWebsite.fact as careers_website \
    FROM \
        UserInterestedInOrgs \
        LEFT JOIN OrgFact_CareersWebsite \
            ON UserInterestedInOrgs.org_id = OrgFact_CareersWebsite.org_id \
    WHERE \
        UserInterestedInOrgs.username = "MBALearnsToCode"')\
    .sort('name')

InterestingOrg_CareersWebsites_DDF.toPandas()

,name,careers_website
0,Adatao,None
1,Adobe,None
2,AirBnB,None
3,Box,Box.com/careers
4,Cloudera,None
5,Facebook,None
6,GitHub,GithHub.com/about/jobs
7,GitHub,jobs.Lever.co/GitHub
8,GitHub,www.linkedin.com/vsearch/j?f_C=1418841
9,Google,None


In [302]:
MyCareerOpportunity_DDF = hc.sql(
    'SELECT \
        CareerOpportunity.org as org, \
        CareerOpportunity.role as role, \
        COLLECT_SET(CareerOpportunity.geog_region) as geog_regions, \
        CareerOpportunity.open as open, \
        CareerOpportunity.url as url, \
        UserCareerOpportunityStatus.status as status \
    FROM \
        UserInterestedInOrgs \
        JOIN CareerOpportunity ON UserInterestedInOrgs.org_id = CareerOpportunity.org_id \
        LEFT JOIN UserCareerOpportunityStatus ON \
            CareerOpportunity.id = UserCareerOpportunityStatus.careeropportunity_id \
    WHERE \
        UserInterestedInOrgs.username = "MBALearnsToCode" OR \
        UserCareerOpportunityStatus.username ="MBALearnsToCode" \
    GROUP BY \
        CareerOpportunity.id, \
        CareerOpportunity.org, \
        CareerOpportunity.role, \
        CareerOpportunity.open, \
        CareerOpportunity.url, \
        UserCareerOpportunityStatus.status \
    ORDER BY \
        CareerOpportunity.org, \
        CareerOpportunity.role, \
        NOT CareerOpportunity.open')

MyCareerOpportunity_DDF.toPandas()

,org,role,geog_regions,open,url,status
0,Adatao,Data Scientist [mid-level],"[San Francisco Bay Area, CA]",True,http://www.linkedin.com/jobs2/view/62346316,None
1,Adobe,Data Scientist [mid-level],"[San Francisco Bay Area, CA]",True,,applied; LinkedIn messaged to recruiter; TODO:...
2,Cloudera,Data Scientist [mid-level],"[San Francisco Bay Area, CA, San Francisco, CA]",True,,None
3,Facebook,Data Scientist [mid-level],"[San Francisco Bay Area, CA]",True,http://www.linkedin.com/jobs2/view/77971276,None
4,GitHub,Data Scientist - User Research [mid-level],"[San Francisco, CA]",True,,None
5,GitHub,Data Scientist [mid-level],"[San Francisco, CA]",True,,Applied; heard nothing
6,InstaCart,Data Scientist [mid-level],"[San Francisco Bay Area, CA]",True,https://www.linkedin.com/jobs2/view/81868859,applied
7,InstaCart,Senior Data Analyst - Marketing [mid-level],"[San Francisco, CA]",True,https://boards.greenhouse.io/instacart/jobs/12...,None
8,InstaCart,Senior Data Analyst - Product [mid-level],"[San Francisco, CA]",True,https://boards.greenhouse.io/instacart/jobs/57242,applied
9,LendUp,Data Analyst [mid-level],"[San Francisco, CA]",True,http://boards.greenhouse.io/lendup/jobs/99802,None


# _end._